In [1]:
# Install dash

In [2]:
pip install dash pandas plotly

Note: you may need to restart the kernel to use updated packages.


In [3]:
#Import libraries
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import webbrowser
from threading import Timer

In [4]:
# Load dataset
orders = pd.read_csv('global_superstore_orders.csv', encoding='ISO-8859-1')
returns = pd.read_csv('global_superstore_returns.csv', encoding='ISO-8859-1')

In [5]:
# Prepare data for visualization
orders['Order Date'] = pd.to_datetime(orders['Order Date'], dayfirst=True)
orders['Ship Date'] = pd.to_datetime(orders['Ship Date'], dayfirst=True)

orders['Year'] = orders['Order Date'].dt.year
orders['Month'] = orders['Order Date'].dt.month
orders['Day'] = orders['Order Date'].dt.day
orders['YearMonth'] = orders['Order Date'].dt.to_period('M')

orders = pd.merge(orders, returns, on='Order ID', how='left')
orders['Returned'] = orders['Returned'].apply(lambda x: 'Yes' if pd.notnull(x) else 'No')

monthly_sales = orders.groupby('YearMonth')['Sales'].sum().reset_index()
monthly_profit = orders.groupby('YearMonth')['Profit'].sum().reset_index()
region_sales = orders.groupby('Region')['Sales'].sum().reset_index()
category_profit = orders.groupby('Category')['Profit'].sum().reset_index()
sub_category_sales = orders.groupby(['Category', 'Sub-Category'])['Sales'].sum().reset_index()
segment_sales = orders.groupby('Segment')['Sales'].sum().reset_index()
segment_profit = orders.groupby('Segment')['Profit'].sum().reset_index()

monthly_sales['YearMonth'] = monthly_sales['YearMonth'].dt.to_timestamp()
monthly_profit['YearMonth'] = monthly_profit['YearMonth'].dt.to_timestamp()

In [6]:
# Dashboard layout
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Global Superstore Sales Dashboard", style={'textAlign': 'center'}),

    html.Div([
        html.Div([
            html.Label('Select Year:'),
            dcc.Dropdown(
                id='year-dropdown',
                options=[{'label': 'All', 'value': 'All'}] + [{'label': year, 'value': year} for year in sorted(orders['Year'].unique())],
                value='All',
                clearable=False,
                style={'width': '200px'}
            )
        ], style={'display': 'inline-block', 'margin-right': '20px'}),
        
        html.Div([
            html.Label('Select Category:'),
            dcc.Dropdown(
                id='category-dropdown',
                options=[{'label': 'All', 'value': 'All'}] + [{'label': category, 'value': category} for category in orders['Category'].unique()],
                value='All',
                clearable=False,
                style={'width': '200px'}
            )
        ], style={'display': 'inline-block', 'margin-right': '20px'}),
        
        html.Div([
            html.Label('Select Region:'),
            dcc.Dropdown(
                id='region-dropdown',
                options=[{'label': 'All', 'value': 'All'}] + [{'label': region, 'value': region} for region in orders['Region'].unique()],
                value='All',
                clearable=False,
                style={'width': '200px'}
            )
        ], style={'display': 'inline-block', 'margin-right': '20px'}),
        
        html.Div([
            html.Label('Select Segment:'),
            dcc.Dropdown(
                id='segment-dropdown',
                options=[{'label': 'All', 'value': 'All'}] + [{'label': segment, 'value': segment} for segment in orders['Segment'].unique()],
                value='All',
                clearable=False,
                style={'width': '200px'}
            )
        ], style={'display': 'inline-block'})
    ], style={'display': 'flex', 'justifyContent': 'center', 'alignItems': 'center', 'margin-bottom': '20px'}),

    html.Div([
        html.Div([
            html.H3("Total Sales"),
            html.P(id="total-sales")
        ], className="two columns", style={'textAlign': 'center'}),

        html.Div([
            html.H3("Total Profit"),
            html.P(id="total-profit")
        ], className="two columns", style={'textAlign': 'center'}),

        html.Div([
            html.H3("Orders Returned"),
            html.P(id="orders-returned")
        ], className="two columns", style={'textAlign': 'center'}),

        html.Div([
            html.H3("Total Orders"),
            html.P(id="total-orders")
        ], className="two columns", style={'textAlign': 'center'}),

        html.Div([
            html.H3("Average Sales per Year"),
            html.P(id="avg-sales-per-year")
        ], className="two columns", style={'textAlign': 'center'}),

        html.Div([
            html.H3("Average Profit per Year"),
            html.P(id="avg-profit-per-year")
        ], className="two columns", style={'textAlign': 'center'}),

        html.Div([
            html.H3("Year with Highest Sales"),
            html.P(id="year-highest-sales")
        ], className="two columns", style={'textAlign': 'center'}),

        html.Div([
            html.H3("Year with Lowest Sales"),
            html.P(id="year-lowest-sales")
        ], className="two columns", style={'textAlign': 'center'}),

        html.Div([
            html.H3("Year with Highest Profit"),
            html.P(id="year-highest-profit")
        ], className="two columns", style={'textAlign': 'center'}),

        html.Div([
            html.H3("Year with Lowest Profit"),
            html.P(id="year-lowest-profit")
        ], className="two columns", style={'textAlign': 'center'}),
    ], className="row", style={'display': 'flex', 'justifyContent': 'space-between'}),

    html.Div([
        dcc.Graph(id='monthly-sales'),
        dcc.Graph(id='monthly-profit')
    ], style={'display': 'flex', 'justifyContent': 'space-between'}),

    html.Div([
        dcc.Graph(id='region-sales'),
        dcc.Graph(id='profit-by-category')
    ], style={'display': 'flex', 'justifyContent': 'space-between'}),

    html.Div([
        dcc.Graph(id='sales-by-category'),
        dcc.Graph(id='profit-by-segment')
    ], style={'display': 'flex', 'justifyContent': 'space-between'}),

    html.Div([
        dcc.Graph(id='segment-sales')
    ], style={'display': 'flex', 'justifyContent': 'center'}),

    html.Div([
        dcc.Graph(id='sub-category-sales-region')
    ], style={'display': 'flex', 'justifyContent': 'center'}),

    html.Div([
        dcc.Graph(id='customer-segment-sales-value')
    ], style={'display': 'flex', 'justifyContent': 'center'}),

    html.Div([
        dcc.Graph(id='customer-segment-rfm')
    ], style={'display': 'flex', 'justifyContent': 'center'}),

    html.Div([
        dcc.Graph(id='profit-by-region')
    ], style={'display': 'flex', 'justifyContent': 'center'}),
])

@app.callback(
    [Output('total-sales', 'children'),
     Output('total-profit', 'children'),
     Output('orders-returned', 'children'),  
     Output('total-orders', 'children'),
     Output('avg-sales-per-year', 'children'),  
     Output('avg-profit-per-year', 'children'),  
     Output('year-highest-sales', 'children'),
     Output('year-lowest-sales', 'children'),  
     Output('year-highest-profit', 'children'),  
     Output('year-lowest-profit', 'children'),  
     Output('monthly-sales', 'figure'),
     Output('monthly-profit', 'figure'),
     Output('region-sales', 'figure'),
     Output('profit-by-category', 'figure'),  
     Output('sales-by-category', 'figure'),
     Output('profit-by-segment', 'figure'),
     Output('segment-sales', 'figure'),
     Output('sub-category-sales-region', 'figure'),  
     Output('customer-segment-sales-value', 'figure'),  
     Output('customer-segment-rfm', 'figure'),  
     Output('profit-by-region', 'figure')],  
    [Input('year-dropdown', 'value'),
     Input('category-dropdown', 'value'),
     Input('region-dropdown', 'value'),
     Input('segment-dropdown', 'value')])

def update_dashboard(selected_year, selected_category, selected_region, selected_segment):
    
    filtered_orders = orders.copy()
    if selected_year != 'All':
        filtered_orders = filtered_orders[filtered_orders['Year'] == selected_year]
    if selected_category != 'All':
        filtered_orders = filtered_orders[filtered_orders['Category'] == selected_category]
    if selected_region != 'All':
        filtered_orders = filtered_orders[filtered_orders['Region'] == selected_region]
    if selected_segment != 'All':
        filtered_orders = filtered_orders[filtered_orders['Segment'] == selected_segment]
        
    total_sales = f"${filtered_orders['Sales'].sum():,.2f}"
    total_profit = f"${filtered_orders['Profit'].sum():,.2f}"
    total_orders = filtered_orders['Order ID'].nunique()

    # Average sales and profit per year
    avg_sales_per_year = filtered_orders.groupby('Year')['Sales'].sum().mean()
    avg_profit_per_year = filtered_orders.groupby('Year')['Profit'].sum().mean()

    # Year with highest and lowest sales
    year_sales = filtered_orders.groupby('Year')['Sales'].sum()
    year_highest_sales = year_sales.idxmax()
    year_lowest_sales = year_sales.idxmin()

    # Year with highest and lowest profit
    year_profit = filtered_orders.groupby('Year')['Profit'].sum()
    year_highest_profit = year_profit.idxmax()
    year_lowest_profit = year_profit.idxmin()

    # Total number of returned orders and the return rate
    orders_returned_count = filtered_orders[filtered_orders['Returned'] == 'Yes']['Order ID'].nunique()
    return_rate = (orders_returned_count / total_orders) * 100 if total_orders > 0 else 0
    orders_returned = f"{orders_returned_count} ({return_rate:.2f}%)"

    # Sales and profit trends
    sales_trend = filtered_orders.groupby('YearMonth')['Sales'].sum().reset_index()
    profit_trend = filtered_orders.groupby('YearMonth')['Profit'].sum().reset_index()
    sales_trend['YearMonth'] = sales_trend['YearMonth'].dt.to_timestamp()
    profit_trend['YearMonth'] = profit_trend['YearMonth'].dt.to_timestamp()
    
    # Sales by region
    region_sales_fig = px.bar(region_sales, x='Sales', y='Region', orientation='h', title='Sales by Region')

    # Profit by category
    category_profit_only_filtered = filtered_orders.groupby('Category').agg(
        Total_Profit=('Profit', lambda x: x[x > 0].sum())
    ).reset_index()

    profit_by_category_fig = px.bar(category_profit_only_filtered, 
                                    x='Category', y='Total_Profit', color='Category', 
                                    title='Total Profit by Category',
                                    labels={'Total_Profit': 'Total Profit'})

    # Sales by category
    sales_by_category_fig = px.bar(filtered_orders.groupby('Category')['Sales'].sum().reset_index(),
                                   x='Sales', y='Category', orientation='h', title='Sales by Category')

    # Profit by segment
    segment_profit_filtered = filtered_orders.groupby('Segment')['Profit'].sum().reset_index()
    profit_by_segment_fig = px.bar(segment_profit_filtered, x='Profit', y='Segment', orientation='h', title='Profit by Segment')

    # Sales by segment
    segment_sales_fig = px.pie(segment_sales, values='Sales', names='Segment', title='Sales by Segment')

    # Top 5 Sub-Categories by sales in each region
    sub_category_sales_region = filtered_orders.groupby(['Region', 'Sub-Category'])['Sales'].sum().reset_index()

    top_5_sub_categories_per_region = sub_category_sales_region.groupby('Region', group_keys=False).apply(
        lambda x: x.nlargest(5, 'Sales')).reset_index(drop=True)

    sub_category_sales_region_fig = px.bar(
        top_5_sub_categories_per_region, 
        x='Region', 
        y='Sales', 
        color='Sub-Category', 
        title='Top 5 Sub-Categories by Sales in Each Region (Sorted by Region Sales)',
        labels={'Sales': 'Total Sales', 'Region': 'Region', 'Sub-Category': 'Sub-Category'},
        barmode='stack', 
        color_discrete_sequence=px.colors.qualitative.Set3 
    )

    sub_category_sales_region_fig.update_layout(
        xaxis={'categoryorder': 'total descending'}, 
        yaxis=dict(
            title='Total Sales',
            tick0=0,
            dtick=100000,  
            tickformat="~s"  
        ),
        margin=dict(l=40, r=40, t=40, b=40)  
    )

    # Customer Segments by total sales value
    customer_sales = filtered_orders.groupby('Customer ID')['Sales'].sum().reset_index()
    high_value_threshold = customer_sales['Sales'].quantile(0.75)
    low_value_threshold = customer_sales['Sales'].quantile(0.25)
    customer_sales['Customer Segment'] = pd.cut(
        customer_sales['Sales'],
        bins=[0, low_value_threshold, high_value_threshold, customer_sales['Sales'].max()],
        labels=['Low Value', 'Medium Value', 'High Value']
    )
    customer_segment_distribution = customer_sales['Customer Segment'].value_counts(normalize=True).reset_index()
    customer_segment_distribution.columns = ['Customer Segment', 'Percentage']
    customer_segment_distribution['Percentage'] *= 100 
    customer_segment_sales_value_fig = px.pie(
        customer_segment_distribution, 
        values='Percentage', 
        names='Customer Segment',
        title='Customer Segments Based on Total Sales Value',
        color_discrete_sequence=px.colors.qualitative.Pastel 
    )

    # Customer Segments by RFM analysis
    rfm_table = filtered_orders.groupby('Customer ID').agg({
        'Order Date': lambda x: (filtered_orders['Order Date'].max() - x.max()).days,  # Recency
        'Order ID': 'count',  
        'Sales': 'sum' 
    }).reset_index()

    rfm_table.columns = ['Customer ID', 'Recency', 'Frequency', 'Monetary Value']
    rfm_table['R_Score'] = pd.qcut(rfm_table['Recency'], 5, ['5', '4', '3', '2', '1'])
    rfm_table['F_Score'] = pd.qcut(rfm_table['Frequency'], 5, ['1', '2', '3', '4', '5'])
    rfm_table['M_Score'] = pd.qcut(rfm_table['Monetary Value'], 5, ['1', '2', '3', '4', '5'])
    rfm_table['RFM_Score'] = rfm_table['R_Score'].astype(str) + rfm_table['F_Score'].astype(str) + rfm_table['M_Score'].astype(str)

    rfm_table['Customer Segment'] = rfm_table['RFM_Score'].apply(lambda x: 
                                                             'Best' if x in ['111', '112', '121', '211', '122'] else
                                                             'Loyal' if x.startswith('1') or x.startswith('2') else
                                                             'Potential' if x.startswith('3') else
                                                             'At Risk' if x.startswith('4') else
                                                             'Lost')

    segment_distribution = rfm_table['Customer Segment'].value_counts(normalize=True).reset_index()
    segment_distribution.columns = ['Customer Segment', 'Percentage']
    segment_distribution['Percentage'] *= 100 
    customer_segment_rfm_fig = px.pie(
        segment_distribution, 
        values='Percentage', 
        names='Customer Segment',
        title='Customer Segments Based on RFM Analysis',
        color_discrete_sequence=px.colors.qualitative.Pastel  
    )

    # Most profitable regions
    profit_by_region = filtered_orders.groupby('Region')['Profit'].sum().reset_index()
    profit_by_region = profit_by_region.sort_values(by='Profit', ascending=False)
    profit_by_region_fig = px.bar(profit_by_region, 
                                  x='Profit', 
                                  y='Region', 
                                  orientation='h', 
                                  title='Most Profitable Regions',
                                  color='Region',
                                  labels={'Profit': 'Total Profit', 'Region': 'Region'},
                                  color_discrete_sequence=px.colors.qualitative.Set2)

    return (total_sales, total_profit, orders_returned, total_orders,
            f"${avg_sales_per_year:,.2f}", 
            f"${avg_profit_per_year:,.2f}", 
            str(year_highest_sales),  
            str(year_lowest_sales),  
            str(year_highest_profit), 
            str(year_lowest_profit), 
            px.line(sales_trend, x='YearMonth', y='Sales', title='Monthly Sales Over Time'),
            px.line(profit_trend, x='YearMonth', y='Profit', title='Monthly Profit Over Time'),
            region_sales_fig, profit_by_category_fig,  
            sales_by_category_fig, profit_by_segment_fig, 
            segment_sales_fig,
            sub_category_sales_region_fig, 
            customer_segment_sales_value_fig, 
            customer_segment_rfm_fig, 
            profit_by_region_fig) 

def open_browser():
    webbrowser.open_new("http://127.0.0.1:8050/")

if __name__ == '__main__':
    Timer(1, open_browser).start()
    app.run_server(debug=True)
